In [ ]:
# Load required libraries
library(ggplot2)
library(dplyr)
library(tidyr)
library(stringr)
library(data.table)
library(jsonlite)

# Set timeout option
options(timeout = 3600)

# Load input parameters from JSON
json_input <- fromJSON("galaxy_inputs/galaxy_inputs.json")

# Extract parameters from JSON
data_path      <- json_input$data$path
selected_cols  <- json_input$colXchose
plot_mode      <- json_input$plotXmode
filter_keyword <- json_input$keyword
plot_title     <- json_input$title
x_axis_label   <- json_input$x
y_axis_label   <- json_input$y
apply_log      <- json_input$log
reference_year <- json_input$year

if(!is.null(json_input$plotXsettings)){
    if(json_input$plotXsettings == "pdf" || json_input$plotXsettings == "PDF"){
      pdf_mode <- TRUE
    } else {
      pdf_mode <- FALSE
      list_setting <- trimws(unlist(strsplit(json_input$plotXsettings, ",")))
      dpi_img <- as.numeric(list_setting[1])
      width_img <- as.numeric(list_setting[2])
      height_img <- as.numeric(list_setting[3])
    }
}
pdf_mode <- if (exists("pdf_mode")) pdf_mode else FALSE
dpi_img    <- if (exists("dpi_img")) dpi_img else NULL
width_img  <- if (exists("width_img")) width_img else NULL
height_img <- if (exists("height_img")) height_img else NULL



if (length(filter_keyword) == 0) {
  filter_keyword <- ""
} else {
    filter_keyword <- gsub("\\(", ".", filter_keyword)
    filter_keyword <- gsub(" ", ".", filter_keyword)
}
if (length(reference_year) == 0) {
  reference_year <- ""
}

# Load dataset
raw_data <- fread(data_path)

# Clean column names to ensure uniqueness and valid syntax
colnames(raw_data) <- make.names(names(raw_data), unique = TRUE)
first_col_name <- colnames(raw_data)[1]

# Convert numeric columns (except Year) by replacing commas with dots
clean_data <- raw_data %>%
  mutate(across(
    .cols = -1,
    .fns = ~ as.double(str_replace_all(., ",", "."))
  ))

# Select relevant columns based on keyword or manual selection
if (!is.null(filter_keyword) && filter_keyword != "") {
  selected_data <- clean_data %>%
    select(all_of(first_col_name), matches(filter_keyword))
} else {
  selected_indices <- as.numeric(unlist(strsplit(selected_cols, ",")))
  selected_data <- clean_data %>%
    select(all_of(first_col_name), selected_indices)
}

if (apply_log) {
  data_log <- selected_data
  
  for (colname in colnames(selected_data)[-1]) {
    if (reference_year == "first") {
      ref_row <- selected_data %>% filter(!is.na(.data[[colname]])) %>% slice(1)
    } else if (reference_year == "last") {
      ref_row <- selected_data %>% filter(!is.na(.data[[colname]])) %>% slice_tail(n = 1)
    } else {
      ref_row <- selected_data %>% filter(.data[[first_col_name]] == reference_year)
    }

    ref_value <- pull(ref_row, colname)
    ref_date <- pull(ref_row, first_col_name)

    if (is.na(ref_value) || length(ref_value) == 0) {
      data_log[[paste0("log_", colname)]] <- NA_real_
    } else {
      data_log[[paste0("log_", colname)]] <- log(selected_data[[colname]] / ref_value)
      data_log[[paste0("perc_", colname)]] <- (selected_data[[colname]] / ref_value - 1) * 100

    }

  }

  selected_data <- data_log %>%
    select(all_of(first_col_name), starts_with("log_"))
}


# Reshape data to long format for plotting
long_data <- selected_data %>%
  pivot_longer(cols = -all_of(first_col_name), names_to = "Species", values_to = "Value")


# Generate plots
if (plot_mode == "merged_view") {
  p <- ggplot(long_data, aes(x = .data[[first_col_name]], y = Value, color = Species, group = Species)) +
    geom_point(na.rm = TRUE) +
    geom_line(na.rm = TRUE) +
    labs(
      title = plot_title,
      x = x_axis_label,
      y = y_axis_label
    ) +
    theme_minimal()
  
  if (!reference_year %in% c("last", "first")) {
    p <- p + geom_vline(xintercept = as.numeric(reference_year), linetype = "dashed", color = "black", linewidth = 0.3, alpha = 0.5)
  }

  file_ext <- if (pdf_mode) ".pdf" else ".png"
  output_file <- paste0("outputs/indicator_plot", file_ext)
  
  if (pdf_mode) {
    pdf(output_file)
    print(p)
    dev.off()
  } else {
    args <- list(filename = output_file, plot = p)
    if (!is.null(dpi_img))    args$dpi   <- dpi_img
    if (!is.null(width_img))  { args$width <- width_img; args$units <- "px" }
    if (!is.null(height_img)) { args$height <- height_img; args$units <- "px" }

    try(do.call(ggsave, args), silent = TRUE)
    
    if (!file.exists(output_file)) {
      ggsave(output_file, plot = p)
    }
  }
} else {
  for (species in unique(long_data$Species)) {
    species_data <- filter(long_data, Species == species)
    filename_clean <- str_replace_all(species, "[^a-zA-Z0-9]", "_")
    file_ext <- if (pdf_mode) ".pdf" else ".png"
    output_file <- paste0("outputs/collection/plot_", filename_clean, file_ext)

    p <- ggplot(species_data, aes(x = .data[[first_col_name]], y = Value)) +
      geom_point(na.rm = TRUE) +
      geom_line(color = "blue", na.rm = TRUE) +
      labs(
        title = paste("Trend -", species),
        x = x_axis_label,
        y = y_axis_label
      ) +
      theme_minimal()
      
    if (!reference_year %in% c("last", "first")) {
      p <- p + geom_vline(xintercept = as.numeric(reference_year), linetype = "dashed", color = "black", linewidth = 0.3, alpha = 0.5)
    }
    if (pdf_mode) {
      pdf(output_file, width = ifelse(!is.null(width_img), width_img/96, 8), height = ifelse(!is.null(height_img), height_img/96, 6))
      print(p)
      dev.off()
    } else {
      args <- list(filename = output_file, plot = p)
      if (!is.null(dpi_img))    args$dpi   <- dpi_img
      if (!is.null(width_img))  { args$width <- width_img; args$units <- "px" }
      if (!is.null(height_img)) { args$height <- height_img; args$units <- "px" }

      try(do.call(ggsave, args), silent = TRUE)
    }
    if (!file.exists(output_file)) {
      ggsave(output_file, plot = p)
    }
  }
}